In [1]:
import xml.etree.ElementTree as ET
from pathlib import Path
import datetime as dt
import xmltodict
import pandas as pd
import os
import zipfile

def read_eml(path: Path):

    """Convert EML file to dictionary"""

    try:
        return xmltodict.parse(path.read_text(encoding="utf-8"))

    except UnicodeDecodeError as e:
        print(path.name)
        print(e)
        return None
    
# Get the user's home directory
current_directory = os.getcwd()

ModuleNotFoundError: No module named 'xmltodict'

In [1]:
# HAVE YOU MOVED THE ZIP FILE INTO THE WORKING DIRECTORY??

zip_filename = ''

# Let's proceed. 

# Construct the full path to your file
zip_file_path = os.path.join(current_directory, zip_filename)

# unzipped directory
unzip_directory_name = zip_filename[:-4]

# Create a ZipFile object and extract the contents
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_directory_name)

print('Unzipped to: ' + unzip_directory_name)
print('Now filtering to spreadsheet ...')

os.chdir(unzip_directory_name + '//xml')
file_name = os.listdir()[-1]

f = open(file_name, encoding='utf-8')
text = f.read()
f.close()
temp = xmltodict.parse(text)

print('File opened...')

df = pd.DataFrame()

for PollingDistrict in temp['MediaFeed']['Results']['Election']['Referendum']['Contests']['Contest']['PollingDistricts']['PollingDistrict']:
    temp_dict = {
        'Seat': PollingDistrict['PollingDistrictIdentifier']['Name'],
        'SeatID': PollingDistrict['PollingDistrictIdentifier']['@Id'],
        'State': PollingDistrict['PollingDistrictIdentifier']['StateIdentifier']['@Id'],
        'Enrolment': float(PollingDistrict['Enrolment']['#text']),
        'Yes': float(PollingDistrict['ProposalResults']['Option'][0]['Votes']['#text']),
        'No': float(PollingDistrict['ProposalResults']['Option'][1]['Votes']['#text']),
        'Informal': float(PollingDistrict['ProposalResults']['Informal']['Votes']['#text']),
        'Complete': PollingDistrict['ProposalResults']['@PollingPlacesExpected'] == PollingDistrict['ProposalResults']['@PollingPlacesReturned'],
        'Votes_returned': float(PollingDistrict['ProposalResults']['Total']['Votes']['@Percentage'])
    }
    temp_dict['Informal_pc'] = temp_dict['Informal'] / temp_dict['Enrolment']
    temp_dict['Yes_pc'] = temp_dict['Yes'] / temp_dict['Enrolment']
    temp_dict['No_pc'] = temp_dict['No'] / temp_dict['Enrolment']
    df = pd.concat(
        [df, pd.DataFrame(temp_dict, index=[0])], ignore_index=True
    )

print(df)

NameError: name 'os' is not defined